<a href="https://colab.research.google.com/github/chumingyzx/Kaggle---Multimodal-Single-Cell-Integration/blob/main/cite_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/NeurIPS/Data/'
all_files = os.listdir(folder_path)

In [ ]:
!pip install mudata
!pip install scanpy
!pip install muon
!pip install scvi-tools


In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
from scipy.sparse import load_npz
from scvi.model import TOTALVI


input_path = '/content/drive/MyDrive/NeurIPS/Data/'
features_path = '/content/drive/MyDrive/NeurIPS/Data/'
model_path = '/content/drive/MyDrive/NeurIPS/model/'
sub_path = '/content/drive/MyDrive/NeurIPS/'

In [ ]:
def zscore_rows(x: np.ndarray) -> np.ndarray:
    mean = np.mean(x, axis=1, keepdims=True)
    std = np.std(x, axis=1, keepdims=True)
    return (x - mean) / (std + 1e-8)

In [ ]:
train_df = pd.read_feather(features_path+'train_cite_inputs_id.feather')
test_df  = pd.read_feather(features_path+'test_cite_inputs_id.feather')
train_targets = np.load(features_path+'train_cite_targets.npy')

In [ ]:
train_df

In [ ]:
cite_svd_clr = np.load(features_path+'cite_inputs_svd_clr_200.npy')
train_svd_clr = zscore_rows(cite_svd_clr[:len(train_df)])
test_svd_clr  = zscore_rows(cite_svd_clr[len(train_df):])

In [ ]:
cite_svd_clr

In [ ]:
cite_feats = np.load(features_path+'cite_inputs_raw_important_feats.npy')
train_feats = zscore_rows(cite_feats[:len(train_df)])
test_feats  = zscore_rows(cite_feats[len(train_df):])

In [ ]:
test_feats

In [ ]:
test_cite_inputs = pd.read_hdf(input_path+'test_cite_inputs.h5').reset_index()[['cell_id']]

In [ ]:
#cite_inputs_svd_clr
cite_inputs_svd_clr = np.load(features_path+'cite_inputs_svd_clr_200.npy')
train_cite_svd_clr = cite_inputs_svd_clr[:len(train_df)]
test_cite_svd_clr = cite_inputs_svd_clr[len(train_df):]
train_cite_svd_clr = zscore_rows(train_cite_svd_clr)
test_cite_svd_clr = zscore_rows(test_cite_svd_clr)

df_test_cite_svd_clr = pd.DataFrame(test_cite_svd_clr)
df_test_cite_svd_clr['cell_id'] = test_cite_inputs['cell_id']
test_cite_inputs_id = test_cite_inputs.copy()
test_cite_inputs_id = test_cite_inputs_id.merge(df_test_cite_svd_clr, on=['cell_id'], how='left')
test_cite_inputs_id = test_cite_inputs_id.fillna(0)
test_cite_svd_clr = test_cite_inputs_id.drop(['cell_id'],axis=1).values

#cite_inputs_bio_norm
cite_inputs_bio_norm_svd = np.load(features_path+'cite_inputs_bio_norm_svd_100.npy')
train_cite_inputs_bio_norm_svd = cite_inputs_bio_norm_svd[:len(train_df)]
test_cite_inputs_bio_norm_svd = cite_inputs_bio_norm_svd[len(train_df):]
train_cite_inputs_bio_norm_svd = zscore_rows(train_cite_inputs_bio_norm_svd)
test_cite_inputs_bio_norm_svd = zscore_rows(test_cite_inputs_bio_norm_svd)

df_test_cite_inputs_bio_norm_svd = pd.DataFrame(test_cite_inputs_bio_norm_svd)
df_test_cite_inputs_bio_norm_svd['cell_id'] = test_cite_inputs['cell_id']
test_cite_inputs_id = test_cite_inputs.copy()
test_cite_inputs_id = test_cite_inputs_id.merge(df_test_cite_inputs_bio_norm_svd, on=['cell_id'], how='left')
test_cite_inputs_id = test_cite_inputs_id.fillna(0)
test_cite_inputs_bio_norm_svd = test_cite_inputs_id.drop(['cell_id'],axis=1).values

#cite_inputs_raw_important_feats
cite_inputs_feats = np.load(features_path+'cite_inputs_raw_important_feats.npy')
train_cite_inputs_feats = cite_inputs_feats[:len(train_df)]
test_cite_inputs_feats = cite_inputs_feats[len(train_df):]
train_cite_inputs_feats = zscore_rows(train_cite_inputs_feats)
test_cite_inputs_feats = zscore_rows(test_cite_inputs_feats)

df_test_cite_inputs_feats = pd.DataFrame(test_cite_inputs_feats)
df_test_cite_inputs_feats['cell_id'] = test_cite_inputs['cell_id']
test_cite_inputs_id = test_cite_inputs.copy()
test_cite_inputs_id = test_cite_inputs_id.merge(df_test_cite_inputs_feats, on=['cell_id'], how='left')
test_cite_inputs_id = test_cite_inputs_id.fillna(0)
test_cite_inputs_feats = test_cite_inputs_id.drop(['cell_id'],axis=1).values

#cite_inputs_bio_norm_pca_64
cite_inputs_bio_norm_pca_64 = np.load(features_path+'cite_inputs_bio_norm_pca_64.npy')
train_cite_inputs_bio_norm_pca_64 = cite_inputs_bio_norm_pca_64[:len(train_df)]
test_cite_inputs_bio_norm_pca_64 = cite_inputs_bio_norm_pca_64[len(train_df):]
train_cite_inputs_bio_norm_pca_64 = zscore_rows(train_cite_inputs_bio_norm_pca_64)
test_cite_inputs_bio_norm_pca_64 = zscore_rows(test_cite_inputs_bio_norm_pca_64)

df_test_cite_inputs_bio_norm_pca_64 = pd.DataFrame(test_cite_inputs_bio_norm_pca_64)
df_test_cite_inputs_bio_norm_pca_64['cell_id'] = test_cite_inputs['cell_id']
test_cite_inputs_id = test_cite_inputs.copy()
test_cite_inputs_id = test_cite_inputs_id.merge(df_test_cite_inputs_bio_norm_pca_64, on=['cell_id'], how='left')
test_cite_inputs_id = test_cite_inputs_id.fillna(0)
test_cite_inputs_bio_norm_pca_64 = test_cite_inputs_id.drop(['cell_id'],axis=1).values


In [ ]:
#xgb1
cite_xgb1_svd = np.load(features_path+'cite_xgb1_svd_100.npy')
train_cite_xgb1_svd = cite_xgb1_svd[:len(train_df)]
test_cite_xgb1_svd = cite_xgb1_svd[len(train_df):]
train_cite_xgb1_svd = zscore_rows(train_cite_xgb1_svd)
test_cite_xgb1_svd = zscore_rows(test_cite_xgb1_svd)


#xgb2
cite_xgb2_svd = np.load(features_path+'cite_xgb2_svd_100.npy')
train_cite_xgb2_svd = cite_xgb2_svd[:len(train_df)]
test_cite_xgb2_svd = cite_xgb2_svd[len(train_df):]
train_cite_xgb2_svd = zscore_rows(train_cite_xgb2_svd)
test_cite_xgb2_svd = zscore_rows(test_cite_xgb2_svd)


#xgb3
cite_xgb3_svd = np.load(features_path+'cite_xgb3_svd_100.npy')
train_cite_xgb3_svd = cite_xgb3_svd[:len(train_df)]
test_cite_xgb3_svd = cite_xgb3_svd[len(train_df):]
train_cite_xgb3_svd = zscore_rows(train_cite_xgb3_svd)
test_cite_xgb3_svd = zscore_rows(test_cite_xgb3_svd)

df_test_cite_xgb3_svd = pd.DataFrame(test_cite_xgb3_svd)
df_test_cite_xgb3_svd['cell_id'] = test_cite_inputs['cell_id']
test_cite_inputs_id = test_cite_inputs.copy()
test_cite_inputs_id = test_cite_inputs_id.merge(df_test_cite_xgb3_svd, on=['cell_id'], how='left')
test_cite_inputs_id = test_cite_inputs_id.fillna(0)
test_cite_xgb3_svd = test_cite_inputs_id.drop(['cell_id'],axis=1).values

#xgb4
cite_xgb4_svd = np.load(features_path+'cite_xgb4_svd_100.npy')
train_cite_xgb4_svd = cite_xgb4_svd[:len(train_df)]
test_cite_xgb4_svd = cite_xgb4_svd[len(train_df):]
train_cite_xgb4_svd = zscore_rows(train_cite_xgb4_svd)
test_cite_xgb4_svd = zscore_rows(test_cite_xgb4_svd)

df_test_cite_xgb4_svd = pd.DataFrame(test_cite_xgb4_svd)
df_test_cite_xgb4_svd['cell_id'] = test_cite_inputs['cell_id']
test_cite_inputs_id = test_cite_inputs.copy()
test_cite_inputs_id = test_cite_inputs_id.merge(df_test_cite_xgb4_svd, on=['cell_id'], how='left')
test_cite_inputs_id = test_cite_inputs_id.fillna(0)
test_cite_xgb4_svd = test_cite_inputs_id.drop(['cell_id'],axis=1).values




In [ ]:
train_cite_X = np.concatenate([
                               train_cite_svd_clr,
                               train_cite_inputs_feats,
                               train_cite_inputs_bio_norm_svd,
                               train_cite_inputs_bio_norm_pca_64,
                               train_cite_xgb1_svd,
                               train_cite_xgb2_svd,
                               train_cite_xgb3_svd,
                               train_cite_xgb4_svd,
                                ],axis=1)

test_cite_X = np.concatenate([
                              test_cite_svd_clr,
                              test_cite_inputs_feats,
                              test_cite_inputs_bio_norm_svd,
                              test_cite_inputs_bio_norm_pca_64,
                              test_cite_xgb1_svd,
                              test_cite_xgb2_svd,
                              test_cite_xgb3_svd,
                              test_cite_xgb4_svd,
                                ],axis=1)

train_cite_y = np.load(features_path+'train_cite_targets.npy')


In [ ]:

def zscore(x):
    return (x - np.mean(x, axis=1, keepdims=True)) / np.std(x, axis=1, keepdims=True)

train_cite_y = zscore(train_cite_y)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import numpy as np
import gc

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
BATCH_SIZE = 600
EPOCHS = 100
LR = 0.0005
WEIGHT_DECAY = 0.0001
LR_FACTOR = 0.1
N_SPLITS = 5
torch.manual_seed(SEED)
np.random.seed(SEED)

def correlation_score(y_true, y_pred):
    return np.mean([pearsonr(a, b)[0] for a, b in zip(y_true, y_pred)])

class CITEseqDataset(data.Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]


In [ ]:
class CITEseqTransformerModel(nn.Module):
    def __init__(self, input_dim, d_model=512, nhead=8, num_layers=2, dim_feedforward=1024):
        super().__init__()
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, d_model),
            nn.SiLU(),
            nn.Dropout(0.1),
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=0.1,
            activation='gelu',
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.output = nn.Linear(d_model, 140)

    def forward(self, x):
        x = self.input_proj(x)
        x = x.unsqueeze(1)
        x = self.transformer(x)
        x = x.squeeze(1)
        return self.output(x)


In [ ]:
def make_model_fn(config):
    def model_fn(input_dim):
        return CITEseqTransformerModel(
            input_dim=input_dim,
            d_model=config["d_model"],
            nhead=config["nhead"],
            num_layers=config["num_layers"],
            dim_feedforward=config["dim_feedforward"]
        )
    return model_fn

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for X, y in loader:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X.size(0)
    return total_loss / len(loader.dataset)

def evaluate(model, loader):
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for batch in loader:
            if isinstance(batch, (list, tuple)) and len(batch) == 2:
                X, y = batch
                truths.append(y.numpy())
            else:
                X = batch
            X = X.to(DEVICE)
            out = model(X).cpu().numpy()
            preds.append(out)
    if truths:
        return np.vstack(preds), np.vstack(truths)
    else:
        return np.vstack(preds), None


In [ ]:
def run_kfold(train_df, train_X, train_y, test_df, test_X, model_fn, save_prefix="model"):
    input_dim = train_X.shape[1]
    oof_preds = np.zeros((train_df.shape[0], 140))
    test_preds = np.zeros((test_df.shape[0], 140))
    cv_corr = []

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_df)):
        print(f'Fold {fold}')
        model = model_fn(input_dim).to(DEVICE)

        optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=LR_FACTOR, patience=6)
        criterion = nn.MSELoss()

        train_dataset = CITEseqDataset(train_X[train_idx], train_y[train_idx])
        valid_dataset = CITEseqDataset(train_X[valid_idx], train_y[valid_idx])
        test_dataset = CITEseqDataset(test_X)

        train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        valid_loader = data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

        best_loss = np.inf
        best_model = None

        for epoch in range(EPOCHS):
            train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
            val_preds, val_trues = evaluate(model, valid_loader)
            val_loss = criterion(torch.tensor(val_preds), torch.tensor(val_trues)).item()
            scheduler.step(val_loss)

            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = model.state_dict()
                torch.save(model.state_dict(), os.path.join(model_path, f'cite_{save_prefix}_fold{fold}.pt'))
                es_counter = 0
            else:
                es_counter += 1
                if es_counter >= 10:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        model.load_state_dict(best_model)
        preds, _ = evaluate(model, valid_loader)
        oof_preds[valid_idx] = preds
        fold_corr = correlation_score(val_trues, preds)
        cv_corr.append(fold_corr)
        print(f'Fold {fold} Corr: {fold_corr:.4f}')

        test_fold_preds, _ = evaluate(model, test_loader)
        test_preds += test_fold_preds / N_SPLITS

        if fold == N_SPLITS - 1:
            torch.save(model.state_dict(),  os.path.join(model_path,f"cite_{save_prefix}_final.pt"))
            print("Final model saved as transformer_final_model.pt")

        del model
        gc.collect()
        torch.cuda.empty_cache()

    overall_corr = correlation_score(train_y, oof_preds)
    print(f'OOF Corr: {overall_corr:.4f}')
    print(f'All Fold Corrs: {cv_corr}')
    return oof_preds, test_preds


In [ ]:
import itertools
import os
import json



search_space = {
    'd_model': [256, 512, 768],
    'nhead': [4, 8],
    'num_layers': [2, 3, 4],
    'dim_feedforward': [1024, 2048, 3072, 4096]
}



param_grid = list(itertools.product(*search_space.values()))
param_names = list(search_space.keys())

results_summary = []

for i, param_set in enumerate(param_grid):
    config = dict(zip(param_names, param_set))
    print(f"\n🔍 Running config {i}: {config}")

    save_prefix = f"search_{i}_d{config['d_model']}_h{config['nhead']}_l{config['num_layers']}_ff{config['dim_feedforward']}"
    model_fn = make_model_fn(config)

    oof_preds, sub_preds = run_kfold(
        train_df, train_cite_X, train_cite_y,
        test_df, test_cite_X,
        model_fn=model_fn,
        save_prefix=save_prefix
    )

    np.save(f"cite_{save_prefix}_oof.npy", oof_preds)
    np.save(f"cite_{save_prefix}_sub.npy", sub_preds)

    with open(f"cite_{save_prefix}_config.json", "w") as f:
        json.dump({**config}, f)

    results_summary.append({**config})

with open("cite_all_search_results.json", "w") as f:
    json.dump(results_summary, f, indent=2)

print("\n🎉 All search finished!")


In [ ]:
np.save(os.path.join(sub_path, f"cite_{save_prefix}_oof.npy"), oof_preds)